In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'real-and-fake-face-detection:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F105271%2F250645%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240420%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240420T180850Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1c8d61ab8b6bea74d37700f42c45719427feaf6ccf7847efa9bb8b3899fd4b9df108bd90a53fe2f7a8cb323f3512df72fb7c6825242edbe4171dc56c812e17b49b220be34d61096faa2a42b85787de8bc93edf58568d80a3b627373ba74de55ef0f24e9057a8b85af76dd4cc8ecf34301796c98e709d2bd2098d3aba2a7443d6f88022a8f9f6945b38cdb6abe8231c2e184448ee60091dd05489f320bd7c93806383c295e742ce7c429cf994bee664d2e88362a714238d83ff497b6412e00621fc0d17cae3343ffa39207be9c8cb08b660e0a1b4814082860b9432fc76f4a72b612857c32958ad2129ecf96a71a830aedd0f12f2b4455e17d20ece8783e2962e,140k-real-and-fake-faces:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F501529%2F939937%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240420%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240420T180850Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D86ea2f3592b5a5bba14b9ea21c9b3f95f696063540d3b7b816f71a912e3abb368a70dbf96307df6d36e405d65f16f0959e11e3835d920a315b9d9f3b3ccc7e52149fe383c82500907a9abd3e8db5aa4af0d5d9c35cf80d14467f7f9bf905c1e588aad6ce69989fdb3941194a9379da0a909d86a0cd30cbbfa9b54604362f3057627f576bcc80c8b91da211665f9808af9f933eafe849106e74fdcfc9115796ec707d92d94f3d9bac2b01fa80c45540f2421602edbf7339167c79bcecdc6b6b8f19adc588405e9b499757fe3638c11b0915aaa96d008fb5d115fa5c91c3c19bdb4e38ab131a71019453e54597dab2199693bb253cbd6072bed30a9bfc384fbb54,deepfake-and-real-images:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1909705%2F3134515%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240420%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240420T180850Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D5e54f8b17ec27cab2c99431a4a4e997889739ab2b598f2447c275eae97bc56c273dd320d678f48019f435faf92996d0233fa86a58b139923ee59fb2a7b36a3fba61a9ae2efc4caec1b72f5d030d0ffe7ff2f8d7bb307085d5836b9f3f45a0ddc4e9acf4e5a82f32f259b62dd90e7b81fbbdd77097ad38636b5e60b763287e67b8be83e7a7d278bb0568541f6b82eb4fed97a41440502b036a917cea5aae348894754cdbd9e42e7a58bdf75e4f5589738af5c002a3fcf16c3778b521916cc69bb730be28396a4ab3667a05d733d06dbd533cb67f2690c68e023f4eb2bf759f36e397745f49be7f1a0cb11498e5008cc290b26f5e60b38944f229e7141fbe169e1,hardfakevsrealfaces:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1913588%2F3142794%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240420%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240420T180850Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D7d866d1cf03c7beb75882c4d5aaa1e57bd9d86dedbab25efc28b804c4e381bd8bc6a30255729e6e109c2ac3dd37501e099357356f59f59d7e5b60964c0c39deec149eb60fbf6145ec03d0228906ab18abde1b9580c54f3f2b8b7ab35a6d92130eec7e96aa36060a99ee7e232ac3b212d21d992f57c7ce841bc69126beefe921e6ea69af9e00313b8f227534f377bab75ced36c200e55cc8d1ccf7f9a86a6411575c725c419ee099d0c260e41fb21065491f8fa74a66d21f9b3108ec4b90808a2f740108999ed71e61dc0536e953b0de481f94f08516fcb73e4025f9e91e3aee2e1c74c3d5da5bd85f3a570968b7a7a69f3fdcb25b48843934c86d0aa7d296fa6,real-vs-ai-generated-faces-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4270700%2F7353626%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240420%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240420T180850Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1bf5b1c3b80ee0db05ee389a89a757ba246ea49b3632ea8b37957490bdd3ff0bbca88e6e280cce5c295aa544b2632f3e133e5e90671173bdfe7e6a5a8fe945bfd103bbbb42b62ee797550f4e1b7c8859dd2dcb207bae7c6507cbbcd59664000bb3ffdca91fc5f0cc82578b9db1045903a4f7449be64a2d944cc55d1464c7f749aaf49c8b7f6fecdf7a772bce6f11a5f7da0ce9e0e22fe9918896a2e9d7bf7febd8f1b9af3b953956cb4864c9d3805c7077a2d91216594b5654efc7d3424118f049648f4a421eaeeb381719680cc2b272a224928f836f258a43bb25a3b503eaf1a6c2f0a5f0eba06b5c91110c017698033ea161ab29f64df9f4e2b165c29962b6'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 452107760 bytes downloaded
Downloaded and uncompressed: real-and-fake-face-detection
[==================================================] 4024555718 bytes downloaded
Downloaded and uncompressed: 140k-real-and-fake-faces
[==================================================] 1808722718 bytes downloaded
Downloaded and uncompressed: deepfake-and-real-images
[==================================================] 16043083 bytes downloaded
Downloaded and uncompressed: hardfakevsrealfaces
[==================================================] 7717595535 bytes downloaded
Downloaded and uncompressed: real-vs-ai-generated-faces-dataset
Data source import complete.


In [ ]:
import os
import tensorflow as tf
from matplotlib import pyplot as plt
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
from keras.models import load_model
from keras.preprocessing import image
from keras.applications import MobileNetV2
from keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
from keras.preprocessing.image import ImageDataGenerator
import csv

In [ ]:
data_csv='/kaggle/input/dataset/dataset.csv'
data1_dir = '/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake'
data2_dir = '/kaggle/input/deepfake-and-real-images/Dataset'
data3_dir = '/kaggle/input/hardfakevsrealfaces'
data4_dir = '/kaggle/input/real-and-fake-face-detection/real_and_fake_face'
data5_dir = '/kaggle/input/real-vs-ai-generated-faces-dataset/dataset/dataset'

In [ ]:
os.listdir(data5_dir)

['test', 'train', 'validate']

In [ ]:

if os.path.exists(data_csv):
    print('exists')
else:

    data_dirs = [ data5_dir, data4_dir]

    # Define the path for the CSV file
    csv_file = 'dataset1.csv'
    data_csv=csv_file

    # Define the class mapping
    class_mapping = {
        'fake': 'Fake',
        'real': 'Real',
        'Fake': 'Fake',
        'Real': 'Real',
        'training_fake': 'Fake',
        'training_real': 'Real',
        '0':'Real',
        '1':'Fake'
    }

    # Open the CSV file in write mode
    with open(csv_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['image', 'label'])  # Write the header row

        # Iterate through the directories in the dataset directory
        for data_dir in data_dirs:
            for root, dirs, files in os.walk(data_dir):
                for file in files:
                    if file.endswith('.jpg') or file.endswith('.png'):  # Adjust file extensions as needed
                        # Get the image path
                        image_path = os.path.join(root, file)

                        # Extract the label from the parent folder name
                        label = os.path.basename(root)

                        # Apply the mapping to the label
                        mapped_label = class_mapping.get(label, label)

                        # Write the image path and mapped label to the CSV file
                        writer.writerow([image_path, mapped_label])


In [ ]:
target_size = (150, 150)

In [ ]:
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator

# Load the CSV file into a DataFrame
df = pd.read_csv(data_csv)

# Create an ImageDataGenerator instance with rescaling and other desired preprocessing options
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # Split the data into train/validation sets
)

In [ ]:
# Create a data generator from the DataFrame using flow_from_dataframe()
data_generator = datagen.flow_from_dataframe(
    dataframe=df,
    x_col='image',         # Column containing the image paths
    y_col='label',         # Column containing the labels
    target_size=target_size,
    batch_size=32,
    class_mode='binary',  # Use 'categorical' for multi-class classification
    shuffle=True,           # Shuffle the data
    seed=42,                # Set random seed for reproducibility
    subset='training'       # Use 'training' subset for training data
)

Found 98396 validated image filenames belonging to 2 classes.


In [ ]:
# Create a validation data generator
validation_generator = datagen.flow_from_dataframe(
    dataframe=df,
    x_col='image',
    y_col='label',
    target_size=target_size,
    batch_size=32,
    class_mode='binary',
    shuffle=True,
    seed=42,
    subset='validation'  # Use 'validation' subset for validation data
)

Found 24599 validated image filenames belonging to 2 classes.


In [ ]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Create a CNN model
model = Sequential()

# Block 1
model.add(Conv2D(32, (3, 3), activation='relu', padding='same',input_shape=(target_size[0],target_size[1], 3)))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# Block 2
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# Block 3
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# Block 4
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# # Block 5
# model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
# model.add(MaxPooling2D((2, 2), strides=(2, 2)))


# Classification block
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(512, activation='relu'))

model.add(Dense(1, activation='sigmoid'))


In [ ]:
model.compile('adam',loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
history = model.fit(data_generator,batch_size=32,epochs=10,validation_data=validation_generator)

Epoch 1/10
 883/3075 [=======>......................] - ETA: 1:22:00 - loss: 0.5695 - accuracy: 0.7012

In [ ]:
model.save('deepfake_detector.h5')

In [ ]:
from keras.models import load_model
model = load_model('deepfake_detector.h5')

In [ ]:
model.build

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from huggingface_hub import from_pretrained_keras

reloaded_model = from_pretrained_keras("Pinchu05/DeepFake_Detection")

In [ ]:
reloaded_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'AUC', 'Precision', 'Recall', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])
history1 = reloaded_model.fit(data_generator,batch_size=32,epochs=2,validation_data=validation_generator)

Epoch 1/2
3075/3075 [==============================] - 7116s 2s/step - loss: 0.0747 - accuracy: 0.9718 - auc: 0.9956 - precision: 0.9699 - recall: 0.9819 - true_positives: 56046.0000 - true_negatives: 39575.0000 - false_positives: 1740.0000 - false_negatives: 1035.0000 - val_loss: 0.1617 - val_accuracy: 0.9469 - val_auc: 0.9833 - val_precision: 0.9486 - val_recall: 0.9586 - val_true_positives: 13421.0000 - val_true_negatives: 9872.0000 - val_false_positives: 727.0000 - val_false_negatives: 579.0000
Epoch 2/2
2520/3075 [=======================>......] - ETA: 19:57 - loss: 0.0639 - accuracy: 0.9767 - auc: 0.9967 - precision: 0.9757 - recall: 0.9843 - true_positives: 45957.0000 - true_negatives: 32802.0000 - false_positives: 1146.0000 - false_negatives: 731.0000